In [2]:
from main import *
from pydantic import ValidationError
from pprint import pprint

github_client = Github(auth=Auth.Token(os.environ["GITHUB_TOKEN"]))
repo = github_client.get_repo("ktrnka/company-detective")

model = get_model("github", "gpt-4o")

try:
    result = review_pull_request(model, repo, repo.get_pull(44), use_base_readme=True)
    print(result)
except ValidationError as e:
    pprint(e.json)


should_update=True reason="The pull request modifies the behavior of Google Play scraping to respect the 'num_reviews' parameter, which is a documented feature in the README. The README currently states that the scraper fetches up to 100 recent reviews, but it does not mention that the 'num_reviews' parameter is now respected. This change should be reflected in the README to ensure accuracy." updated_readme="# Company Detective\n\nThis project summarizes publicly available information about a company. It leverages various APIs to gather and analyze data, providing a comprehensive overview of the target company.\n\nLive site: https://ktrnka.github.io/company-detective\n\n![System diagram](system_diagram.png)\n\n## Features\n\n- Multiple information sources including Crunchbase, news articles, and company websites\n- Utilizes AI to analyze and summarize information\n- Configured via Airtable\n- Google Analytics integration for tracking user interactions and site performance.\n\n## Prereq

In [ ]:
# APIStatusError: Error code: 413 - {'error': {'code': 'tokens_limit_reached', 'message': 'Request body too large for gpt-4o-mini model. Max size: 8000 tokens.', 'details': 'Request body too large for gpt-4o-mini model. Max size: 8000 tokens.'}}



# o1
# BadRequestError: Error code: 400 - {'error': {'code': 'BadRequest', 'message': 'Model o1 is enabled only for api versions 2024-12-01-preview and later'}}
# After updating:
# BadRequestError: Error code: 400 - {'error': {'message': "Unsupported value: 'messages[0].role' does not support 'system' with this model.", 'type': 'invalid_request_error', 'param': 'messages[0].role', 'code': 'unsupported_value'}}

# Llama-3.3-70B-Instruct
# NotFoundError: NOT FOUND

# Meta-Llama-3.1-70B-Instruct
# NotFoundError: NOT FOUND

In [5]:
print(f"""
Should update: {result.should_update}

Reason:
{result.reason}

Updated README:
{result.updated_readme}
""")


Should update: False

Reason:
The current README is comprehensive and up-to-date. The pull request changes are primarily focused on internal code modifications and don't introduce any new features or significant changes that would require updates to the README. The existing README already covers the project's purpose, features, prerequisites, installation instructions, and usage information adequately.

Updated README:
None



In [7]:
# Show the readme diff
import difflib

if result.should_update:
    diff = difflib.unified_diff(
        repo.get_readme().decoded_content.decode("utf-8").splitlines(),
        result.updated_readme.splitlines(),
    )
    for line in diff:
        print(line)
